In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import os

In [ ]:
start_date = "2021-11-10"
end_date = "2021-12-10"
date_list = pd.date_range(start_date, end_date).to_list()

In [ ]:
precip = xr.open_dataset(f'climate_data/ppt_{start_date}-{end_date}.nc')
tmean = xr.open_dataset(f'climate_data/tmean_{start_date}-{end_date}.nc')

In [ ]:
snow_threshold = 0 # degrees celsius
mf = 5 # melt factor, mm/degree celsius that snow melts when T>snow threshold

# empty variable to append dataarrays to
nsd = []
nsa = []

#first day of precipitation and temperature
first_day_precip = precip['ppt'].sel(date=date_list[0])
first_day_temp = tmean['tmean'].sel(date=date_list[0])

# first day snow accumulation
first_day_snowfall = np.where(first_day_temp < snow_threshold, first_day_precip, 0)

# the first day of net snow accumulation into a data array
net_snow_da = xr.DataArray(first_day_snowfall,
                            dims=['lon', 'lat'],
                            coords={'lat': precip.lat.values,
                                    'lon': precip.lon.values})

# add date
net_snow_da = net_snow_da.expand_dims(dim = 'date')
net_snow_da.coords['date'] = ('date', [date_list[0]])

# assumption: initial snow is 0 across basin (likely flawed)
# change in snow is just equal to the accumulation of that day
nsd.append(net_snow_da)
nsa.append(net_snow_da)

# iterate through the rest of the dates
for date in date_list[1:]:
    # get precip and temp for the current day in the loop
    date_precip = precip['ppt'].sel(date=date)
    date_temp = tmean['tmean'].sel(date=date)
    
    # get net snowfall/melt for the day
    date_snowfall = np.where(date_temp < snow_threshold, date_precip, 0)
    date_snowmelt = np.where(date_temp > snow_threshold, mf*(date_temp - snow_threshold), 0)
    net_snow_per_day = date_snowfall - date_snowmelt
    
    # get accumulated snow so far
    net_snow_total = net_snow_da + net_snow_per_day
    
    # find negative values (can't have negative snow)
    net_snow_idx = (net_snow_total < 0)
    net_snow_total = xr.where(net_snow_idx, 0, net_snow_total)
    
    # calculate snow today - snow yesterday to get daily change in snow
    net_snow_change = net_snow_total - net_snow_da
    
    # record daily snow change to data array
    deltasnow_perday = xr.DataArray(net_snow_change,
                            dims=['date', 'lon', 'lat'],
                            coords={'date': [date],
                                    'lat': precip.lat.values,
                                    'lon': precip.lon.values})
    
    # record total snow accumulation to data array
    net_snow_da = xr.DataArray(net_snow_total,
                            dims=['date', 'lon', 'lat'],
                            coords={'date': [date],
                                    'lat': precip.lat.values,
                                    'lon': precip.lon.values})
    
    # append net snow delta and net snow accumulation
    nsd.append(deltasnow_perday)
    nsa.append(net_snow_da)
    

# make netcdf of daily snow change (which would add to or subtract from runoff)
daily_snow_change = xr.concat(nsd, 'date')
daily_snow_change.to_netcdf(path=os.path.join('climate_data/', f'snow_chage_{start_date}-{end_date}.nc'))

# make netcdf of daily snow pack
# not sure if this is necessary... but i accidentally originally did it this way
final_snow = xr.concat(nsa, 'date')
outstring = os.path.join('climate_data/', f'snow_{start_date}-{end_date}.nc')

final_snow.to_netcdf(path=outstring)

In [ ]:
final_snow.isel(date=slice(0, 31, 1)).plot.imshow(col = 'date', col_wrap = 3, vmin = 0, vmax = 20)

In [ ]:
daily_snow_change.isel(date=slice(0, 31, 1)).plot.imshow(col = 'date', col_wrap = 3)